# Introduction à PyTorch : AutoGrad

----

Cette partie du cours permet juste de se servir de la feature autoGrad. Mais qu'est-ce c'est que l'autoGrad ???

L'autoGrad est simplement une feature qui permet de calculer directement les matrices jacobiennes issues des opérations sur les tenseurs avec ou sans paramètres. En fait c'est simplement un outil qui nous permet d'obtenir la matrice des gradients pour appliquer notre optimiseur par derrière pour la partie back-propagation ! Tout est géré automatiquement donc aucun calculs n'est nécessaire.

En ce qui concerne son utilisation, le but est de suivre les entrées du réseau pour y vérifier toutes les opérations. A la sortie du réseau nous avons ainsi la sortie que l'on veut comparer à la fonction de coût. Ainsi le but est de bien entendu trouver le gradient entre les paramètres et la fonction de coût. La règle de la chaîne nous fait ainsi remonter toutes les opérations du réseau et son application par la fonction de coût. Le calcul du gradient se fera ainsi à la toute fin lors de la sortie du réseau et après l'application de la fonction de coût.

In [ ]:
import tor

Pour utiliser la feature d'autoGrad, on va devoir spécifier quels tenseurs on veut suivre pour calculer son gradient au fur et à mesure des opérations qui lui sont appliqués :

In [ ]:
# Création du tenseur qu'on veut suivre :

x = torch.tensor([1., 2., 3.], requires_grad=True) # Rajouter requires_grad=True permet de suivre le tenseur x. On peut alors observer son gradient par rapport à d'autres tenseurs.

y = x + 2 # y est un tenseur qui dépend de x et qu'on veut suivre. Ainsi on peut observer un attribut de y, son gradient par rapport à x.

print(x)
print(y)

print(y.grad_fn) # y a été créé par une opération, on peut donc observer l'opération qui a créé y.
# L'opération en question (AddBackward) a été créée par l'addition de x et 2 et est visible à l'adresse spécifiée.

# Maintenant que se passe-t-il quand on change d'opération ?

z = y * y * 3

print(z) # On observe alors un nouveau gradient function nommé MulBackward0, créé par la multiplication de y par y par 3. 

z = z.mean()

print(z) # MeanBackward0 (encore une autre opération suivie)

Maintenant qu'on a effectué tout un tas d'opération qui on pour origine le tenseur x qui est suivi, on peut alors calculer son gradient très facilement par une application de la règle de la chaine :

In [ ]:
print(x.grad) # On observe que le gradient de x par rapport à z est None, car on n'a pas encore fait de backward sur z.

z.backward() # On calcule le gradient de z par rapport à x.

print(x.grad) # On observe le gradient de x par rapport à z avec une application de la règle de la chaine pour remonter toutes les opérations précédentes.

<u>Remarque importante</u> : On ne peut pas effectuer deux fois l'opération du calcul de gradient, donc il faut bien faire attention de bien effectuer cette opération une fois que tous les calculs sont à priori terminés et à gérer de façon intelligente pour pas qu'il s'exécute deux fois ou plus... C'est ce qui arrivera notamment lorsque l'on itère sur plusieurs épochs sans vider le gradient, on accumulera ainsi les gradients de chaque épochs, faussant ainsi lourdement le résultat.

----

## Retirer les tenseurs de l'autoGrad et remarque :

Maintenant que l'on a vu comment suivre des tenseurs, il faut bien comprendre que ce que l'on veut suivre c'est bien entendu <u>**les paramètres**</u> car ce sont eux que l'on veut regarder pour le gradient de la fonction de coût. L'idée est alors que l'on suit les paramètres durant un epoch, on réinitialise le gradient et on sort les paramètres de l'autoGrad. En effet à chaque fin d'epoch, on veut mettre à jour les paramètres. Cependant cette opération ne rentre surtout pas en compte dans le processus de calcul de gradient, c'est pourquoi on veut retirer les paramètres de l'autoGrad à chaque fois et les remettre ensuite.

In [ ]:
# Nous allons regarder deux méthodes en particulier :

# Utilisation de 'x.require_grad_(false)' :

x = torch.randn(3, requires_grad=True)
print(x.requires_grad) # True
x.requires_grad_(False)
print(x.requires_grad) # False

# Deuxième méthode plus propre :

x = torch.randn(3, requires_grad=True)

with torch.no_grad(): # On désactive le gradient pour x.
    print((x**2).requires_grad) # False
    
# with torch.no_grad() permet de désactiver le gradient pour un bloc de code. Cela en revanche ne change pas le gradient de x mais désactive le gradient pour le bloc de code.
# Ainsi si on avait juste mis x, on aurait eu True, car x a toujours son gradient activé.

----

## Application à l'implémentation d'un modèle de régression linéaire :

On veut maintenant appliquer ce qu'on a vu à un modèle de régression linéaire. On veut ici approximer la fonction $ f : x → 2x $

On va donc partir du modèle linéaire $ f : x → w*x + b $ (ici on fixe directement b à 0 pour simplifier...)

In [ ]:
# On prend des données très simple de X et Y :

X = torch.tensor([1,2,3,4,5,6,7,8], dtype=torch.float32)
Y = torch.tensor([2,4,6,8,10,12,14,16], dtype=torch.float32) # Notre modèle va devoir prédire Y à partir de X.

W = torch.tensor(0.0, dtype = torch.float32, requires_grad=True) # On rajoute bien entendu l'autograd pour W car on veut faire une rétropropagation par la suite.


In [ ]:
# On veut maintenant préciser les fonctions utiles à notre modèle :

# Notre modèle est très simple, il s'agit d'une fonction linéaire, donc on va juste faire une multiplication pour avoir notre sortie après une propagation.
def forward(x):
    return W * x

# On veut maintenant la fonction loss pour calculer l'erreur de notre modèle.
def loss(y, y_predicted):
    return ((y_predicted - y)**2).mean() 


In [ ]:
# Pour l'instant on a nos paramètres à 0, on a donc un résultat de f(x) = 0 pour tout x...
# On va alors commencer l'entraînement avec la définition de nos paramètres :

learning_rate = 0.01
n_epoch = 100


for epoch in range(n_epoch):

    y_pred = forward(X) # On calcule la prédiction de notre modèle.
    
    l = loss(Y, y_pred) # On calcule la loss de notre modèle.
    
    # Maintenant que l'on a tout ça, on va pouvoir faire la rétropropagation :

    l.backward() # On calcule le gradient de l par rapport à W.

    # Vient ensuite la mise à jour des poids (sans autoGrad du coup !!!) :
    
    with torch.no_grad():
        W -= learning_rate * W.grad # On met à jour W.
        
    # On remet à 0 le gradient de W pour la prochaine itération :
    W.requires_grad_(True) # On réactive le gradient de W pour la prochaine itération.
    W.grad.zero_()
    
    # On affiche la loss à chaque 10 itérations :
    
    if (epoch+1) % 10 == 0:
        print(f'epoch {epoch+1}: w = {W:.3f}, loss = {l:.8f}')
        
# Résultat de notre entraînement :
print(f'Prediction after training: f(5) = {forward(5):.3f}')

<u>Remarque</u> : Petite remarque au niveau du code, on peut pas faire $W = W - learning\_rate*W.grad$ sinon on change la variable W pour qu'elle perde son statut de require = True et dans ce cas elle ne peut pas contenir d'élément 'grad' ce qui cause une erreur.
Le fait de faire l'opération $ W -= learning\_rate*W.grad$ en revanche modifie bien W sans pour autant retirer le statut de require = True ce qui est plus avantageux (c'est sombre comme feature...)

----

## Conclusion

Voilà ce qui concerne l'utilisation de l'autoGrad qui est extrêmement utile pour calculer les gradients pour pouvoir les utiliser dans les algorithmes d'optimisation. La manière dont sera calculée le gradient sera sensiblement toujours pareille. En effet, la boucle utilisée pour notre descente de gradient est un exemple à peine simplifié de ce que l'on va implémenter dans les prochains cours pour créer des réseaux de neurones.

*Remarque très importante* :

Connaître l'autograd est utile pour comprendre un peu mieux l'exécution d'une boucle d'entraînement PyTorch car en effet, on définit toujours soi-même la boucle. **CEPENDANT**, son utilisation dans un réseau de neurones est extrêmement simplifiée (les poids sont initialisés avec un suivi automatique et on a juste besoin d'appeler la fonction de backward en fait). C'est pourquoi connaître l'autograd peut sembler un peu inutile. Or ce n'est très loin d'être vrai car beaucoup de travaux de recherches ou de développement d'outils en science des données et ML se font avec non pas des réseaux mais des algorithmes d'optimisations numériques itératifs (même genre qu'une descente de gradient) et l'implémentation s'avère compliqué sans moyen de calculer les gradients. C'est pourquoi nombres de ces outils sont implémentés en PyTorch pour avoir accès à l'autograd. 